#

## Scraping.ipynb

---

Notebook tem como objetivo extrair dados de partidas do site https://football.esportsbattle.com/en/ e salvar em um arquivo .csv

In [107]:
import pandas as pd
import requests, time, random, json
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from datetime import datetime, timedelta
from urllib3.util.retry import Retry



##### gerar_url()

Função que recebe uma data de referência e gera uma lista de URLs que busca os torneios

In [108]:
def gerar_url(data_inicial, data_final = datetime.now()):

    datas = [
        [(data_inicial + timedelta(days=n)).year,
        (data_inicial + timedelta(days=n)).month,
        (data_inicial + timedelta(days=n)).day]
        for n in range((data_final - data_inicial).days + 1)
    ]   

    datas_url = []
    pag = 1 
    for i in range(len(datas)):
        ano = datas[i][0]
        mes = datas[i][1]
        dia = datas[i][2]
        if mes in [1,3,5,7,8,10,12] and dia == 31:
            datas_url.append(f"https://football.esportsbattle.com/api/tournaments?page={pag}&dateFrom={ano}%2F{mes}%2F{dia}+11%3A00&dateTo={ano}%2F{mes}%2F{1}+02%3A59")
        elif mes == 2 and dia == 28:
            datas_url.append(f"https://football.esportsbattle.com/api/tournaments?page={pag}&dateFrom={ano}%2F{mes}%2F{dia}+11%3A00&dateTo={ano}%2F{mes}%2F{1}+02%3A59")
        else:
            datas_url.append(f"https://football.esportsbattle.com/api/tournaments?page={pag}&dateFrom={ano}%2F{mes}%2F{dia}+11%3A00&dateTo={ano}%2F{mes}%2F{dia+1}+02%3A59")

    print(f"{len(datas_url)} URLs geradas")
    return datas_url

In [109]:
data = gerar_url(datetime(2025,1,1), datetime(2025,1,2))

2 URLs geradas


In [110]:
def pega_torneios(datas_url):
    lista_torneios = []  # Lista de todos os torneios disponíveis por página
    print('Extraindo Torneios...')
    for url in tqdm(datas_url):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Lança erro se status != 200
            json_torneios = response.json()

            paginas = json_torneios.get('totalPages', 0)
            torneios = json_torneios.get('tournaments', [])

            for pag in range(paginas):
                for torneio in torneios:
                    lista_torneios.append([pag + 1, torneio])

        except Exception as e:
            print(f"[AVISO] Erro ao processar URL: {url}")
            print(f"Motivo: {e}\n")
            continue

    print(f"{len(lista_torneios)} torneios gerados.")
    return lista_torneios


##### Função que pega todas as partidas por torneio

In [111]:
def pega_partidas(lista_torneios):
    
    print('Extraindo partidas...')
    lista_partidas = []
    for i in tqdm(range(len(lista_torneios))):
        # pag = lista_torneios[i][0]
        id_torneio = lista_torneios[i][1]['id']

        lista_partidas.append(f'https://football.esportsbattle.com/api/tournaments/{id_torneio}/matches')

    print(f"{len(lista_partidas)} partidas geradas")
    return lista_partidas

### Configurações

In [112]:
# ---- sessão global robusta (1x por processo) ----
_session = requests.Session()
_session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json",
    "Connection": "keep-alive",
    "Referer": "https://football.esportsbattle.com/"
})
_retry = Retry(
    total=5,                      # nº máximo de tentativas
    backoff_factor=0.7,           # 0.7s, 1.4s, 2.8s, 5.6s, ...
    status_forcelist=(429, 500, 502, 503, 504),
    allowed_methods=["GET"],
    raise_on_status=False
)
_adapter = HTTPAdapter(max_retries=_retry, pool_connections=20, pool_maxsize=50)
_session.mount("http://", _adapter)
_session.mount("https://", _adapter)

In [113]:
def _get_json(url, timeout=(5, 20), tries_extra=3):
    """GET + JSON com proteção a desconexão sem resposta."""
    for t in range(tries_extra):
        try:
            r = _session.get(url, timeout=timeout)
            if r.status_code != 200:
                # algumas APIs retornam HTML de erro; log mínimo e retry
                raise requests.HTTPError(f"status={r.status_code}")
            # às vezes a resposta vem vazia/HTML e quebra .json()
            return r.json()
        except (requests.exceptions.JSONDecodeError,
                requests.exceptions.ConnectionError,
                requests.exceptions.ReadTimeout,
                requests.exceptions.ChunkedEncodingError,
                requests.exceptions.HTTPError) as e:
            # backoff exponencial com jitter
            wait = (0.5 * (2 ** t)) + random.uniform(0, 0.4)
            print(f"[AVISO] Falha em {url} ({type(e).__name__}). Tentando de novo em {wait:.1f}s...")
            time.sleep(wait)
    # desistiu desta URL
    print(f"[ERRO] Desisti de {url}")
    return None

In [ ]:
def pega_resultados(lista_partidas):

    print('Extraindo resultados...')
    # Aceitar string única também
    if isinstance(lista_partidas, str):
        lista_partidas = [lista_partidas]

    dfs = []
    for url in tqdm(lista_partidas):
        js = _get_json(url)
        if not js:
            continue

        # endpoint as vezes é lista ou dict com 'matches'
        if isinstance(js, dict) and "matches" in js:
            payload = js["matches"]
        else:
            payload = js

        if not payload:
            continue

        df_json = pd.json_normalize(payload)
        dfs.append(df_json)

        # opcional: pequeno intervalo pra não sobrecarregar servidor
        time.sleep(0.15)

    if dfs:
        return pd.concat(dfs, ignore_index=True)
    
    return pd.DataFrame()

In [115]:
def gerar_base(data_inicial, data_final = datetime.now()):
    urls = gerar_url(data_inicial, data_final)
    torneios = pega_torneios(urls)
    partidas = pega_partidas(torneios)
    jogos = pega_resultados(partidas)


In [ ]:
if __name__ == "__main__":
    gerar_base(datetime(2025,1,1))

In [ ]:
# jogos.to_csv("../../data/raw/base_partidas_raw.csv", index=False, encoding='utf-8')